In [92]:
from keras.applications import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, concatenate, BatchNormalization
from keras import backend as k
import keras
from keras import models
from keras import layers
from keras import optimizers
import tensorflow as tf
import os
import numpy as np

<h1> Challenge 1

<h1> Load Train and validation data


In [93]:
from keras.applications.vgg16 import preprocess_input
currentPath= os.getcwd()
train_dir = os.path.join(currentPath, 'data/train/')
validation_dir = os.path.join(currentPath, 'data/val/')
image_size=224

train_datagen = keras.preprocessing.image.ImageDataGenerator(
      rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,shear_range = 0.2,
        zoom_range = 0.2,
      fill_mode='nearest',preprocessing_function = preprocess_input)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = preprocess_input)

# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 64

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        shuffle=True)


Found 4800 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.


In [94]:
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras import callbacks
reduceLearningRate  = 0.5

# create the base pre-trained model
base_model = VGG16(weights='imagenet', include_top=False,input_shape=(image_size, image_size, 3))


x = base_model.output
Pool1 = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x=Flatten()(x)
Dense1 = Dense(256, activation='relu')(Pool1)
BN1=BatchNormalization()(Dense1)
Dropout1=Dropout(0.5)(BN1)
Dense2 = Dense(32, activation='relu')(Dropout1)
BN2=BatchNormalization()(Dense2)
Dropout2=Dropout(0.5)(BN2)
concat1 = concatenate([Pool1, Dropout2], axis=-1)

predictions = Dense(2, activation='softmax')(concat1)

model = Model(inputs=base_model.input, outputs=predictions)



for i, layer in enumerate(model.layers):
    layer.trainable = True
for layer in model.layers[0:17]:
     layer.trainable = False

for i, layer in enumerate(model.layers):
    print(i, layer.name)
    print (i, layer.trainable)


# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['acc'])
model.summary()


if 'outputschallenge1' not in os.listdir(os.curdir):
    os.mkdir('outputschallenge1')


log_filename = 'outputschallenge1/' + 'SLC' +'_results.csv'

csv_log = callbacks.CSVLogger(log_filename, separator=',', append=True)

checkpoint_filepath = 'outputschallenge1/' + 'model-{epoch:03d}.h5'

checkpoint = callbacks.ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [csv_log, checkpoint]
callbacks_list.append(ReduceLROnPlateau(factor=reduceLearningRate, patience=20,
                                           verbose=True))
callbacks_list.append(EarlyStopping(verbose=True, patience=20))


history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1, callbacks=callbacks_list,shuffle=True)


model_name = 'outputschallenge1/' + 'SLC' + '_model_last'
model.save(model_name)  # creates a HDF5 file 'my_model.h5'
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


0 input_1
0 False
1 block1_conv1
1 False
2 block1_conv2
2 False
3 block1_pool
3 False
4 block2_conv1
4 False
5 block2_conv2
5 False
6 block2_pool
6 False
7 block3_conv1
7 False
8 block3_conv2
8 False
9 block3_conv3
9 False
10 block3_pool
10 False
11 block4_conv1
11 False
12 block4_conv2
12 False
13 block4_conv3
13 False
14 block4_pool
14 False
15 block5_conv1
15 False
16 block5_conv2
16 False
17 block5_conv3
17 True
18 block5_pool
18 True
19 global_average_pooling2d_1
19 True
20 dense_1
20 True
21 batch_normalization_1
21 True
22 dropout_1
22 True
23 dense_2
23 True
24 batch_normalization_2
24 True
25 dropout_2
25 True
26 concatenate_1
26 True
27 dense_3
27 True
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
________________________________

300/300 [==============================] - 80s 266ms/step - loss: 0.3570 - acc: 0.8494 - val_loss: 0.3817 - val_acc: 0.8525

Epoch 00007: val_loss improved from 0.39833 to 0.38168, saving model to outputschallenge1/model-007.h5
Epoch 8/100
300/300 [==============================] - 80s 268ms/step - loss: 0.3410 - acc: 0.8490 - val_loss: 0.4028 - val_acc: 0.8400

Epoch 00008: val_loss did not improve from 0.38168
Epoch 9/100
300/300 [==============================] - 80s 267ms/step - loss: 0.3275 - acc: 0.8558 - val_loss: 0.3666 - val_acc: 0.8517

Epoch 00009: val_loss improved from 0.38168 to 0.36660, saving model to outputschallenge1/model-009.h5
Epoch 10/100
300/300 [==============================] - 82s 275ms/step - loss: 0.3168 - acc: 0.8621 - val_loss: 0.3598 - val_acc: 0.8542

Epoch 00010: val_loss improved from 0.36660 to 0.35978, saving model to outputschallenge1/model-010.h5
Epoch 11/100
300/300 [==============================] - 82s 272ms/step - loss: 0.2988 - acc: 0.8721 - v

<h1> Get the final predictions

In [119]:
model.load_weights('outputschallenge1/model-062.h5')

In [120]:
from keras.models import load_model
currentPath= os.getcwd()
test_dir = os.path.join(currentPath, 'testChallenge1/')
image_size=224

test_datagen = keras.preprocessing.image.ImageDataGenerator()

# Change the batchsize according to your system RAM
test_batchsize = 16

test_generator = test_datagen.flow_from_directory(
    directory=r"testChallenge1/",
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical',
    shuffle=False,
    seed=42)

test_generator.reset()
print (max(test_generator.classes))
#modelfinal = load_model('outputschallenge1/model-062.h5')

pred=model.predict_generator(test_generator,verbose=1,steps=len(test_generator))

#modelfinal.predict(test_generator, batch_size=test_batchsize, verbose=1, steps=None)




Found 1015 images belonging to 1 classes.
0
1015/1015 [==============================] - 15s 15ms/step
